In [9]:
'''기본조건'''

#데이터 손실 확인 함수

def check_missings(raw, test):
    #raw data의 행의 개수 
    raw_row = raw.shape[0] 
    
    #raw data 학번 순 정렬 후 index 변경
    raw = raw.sort_values(by = '학번').set_index('학번')
    
    #test data 학번 순 정렬 후 index 변경
    sort_test = test.sort_values(by = '학번').set_index('학번')
    
    #조 배치 컬럼을 제외하고 raw data와 test data의 모든 값이 같은 지를 확인
    try:
        num_corrects = (sort_test.drop('조 배치', axis = 1) == raw)['이름'].sum()
    except:
        return False
    
    return raw_row == num_corrects

#전원이 배치되었는 지 확인

def check_mapping(test):
    #배치되지 않은 부원 수
    no_matches = test['조 배치'].isna().sum() 
    
    return no_matches == 0

#전체 조가 6~7명으로 구성되었는지 확인

def check_group_num(test):
    group_by_count = test.groupby('조 배치').count()
    
    num_groups = group_by_count.shape[0]
    
    num_six_or_sevens = ((group_by_count['학번'] == 6) | (group_by_count['학번'] == 7)).sum()
    
    return num_groups == num_six_or_sevens

#전원 한개의 조에만 배치되었는지 확인

def check_one_to_one(test):
    group_by_student_id = test.groupby('학번').count()
    
    num_groups = group_by_student_id.shape[0]
    
    num_one_to_ones = (group_by_student_id['이름'] == 1).sum()
    
    return num_groups == num_one_to_ones

'''필수 조건'''

#조장 희망자 배치 확인
def check_leader_prefers(test):
    #전체 조의 수를 세줍니다.
    num_groups = test.groupby('조 배치').count().shape[0]
    
    # 조 배치별로 그룹화 한 뒤, 조장 희망 여부가 '희망'인 경우를 세줍니다.
    leader_counts = test[test['조장 희망 여부'] == '희망'].groupby('조 배치').size()
    
    # 조장 희망자가 1 이상인 조의 비율을 반환합니다.
    return sum(leader_counts >= 1) / num_groups

#조장 가능자 배치 확인
def check_possible_leaders(test):
    #전체 조의 수를 세줍니다.
    num_groups = test.groupby('조 배치').count().shape[0]
    
    # 조장 희망 여부가 '희망' 또는 '상관 없음'인 경우를 선택합니다.
    possible_leaders = test[test['조장 희망 여부'].isin(['희망', '상관 없음'])]
    
    # 선택된 데이터를 조 배치별로 그룹화 하고, 각 그룹의 크기(즉, 조장 가능자의 수)를 계산합니다.
    leader_counts = possible_leaders.groupby('조 배치').size()
    
    # 조장 가능자가 1 이상인 조의 비율을 반환합니다.
    return sum(leader_counts >= 1) / num_groups

#음주 호불호 배치 확인
def check_alcohol_prefers(test):
    groups = test['조 배치'].unique()
    total_groups = len(groups)
    satisfying_groups = 0

    for group in groups:
        group_df = test[test['조 배치'] == group]
        if '호' in group_df['음주 호불호'].values and '불호' in group_df['음주 호불호'].values:
            continue
        satisfying_groups += 1

    return satisfying_groups / total_groups

def calculate_gender_satisfying_ratio(raw, test):
    # raw 데이터의 전체 성별 비율 계산
    total_male_ratio = len(raw[raw['성별'] == '남']) / len(raw)
    total_female_ratio = len(raw[raw['성별'] == '여']) / len(raw)

    groups = test['조 배치'].unique()
    total_groups = len(groups)
    total_score = 0

    for group in groups:
        group_df = test[test['조 배치'] == group]
        male_count = len(group_df[group_df['성별'] == '남'])
        female_count = len(group_df[group_df['성별'] == '여'])

        # 각 조의 성별 비율 계산
        group_male_ratio = male_count / (male_count + female_count)
        group_female_ratio = female_count / (male_count + female_count)

        # 각 조의 성별 비율과 전체 성별 비율의 차이의 절대값을 이용하여 점수 계산
        # 이 점수는 성별 비율이 얼마나 비슷한지를 나타냄 (점수가 높을수록 성비가 비슷함)
        group_score = 1 - (abs(group_male_ratio - total_male_ratio) + abs(group_female_ratio - total_female_ratio)) / 2
        total_score += group_score

    # 전체 조의 평균 점수 반환
    return total_score / total_groups

def calculate_age_preference(row, group):
    #조원 나이대 희망을 가지고 계산
    preference = row['조원 나이대 희망']
    
    #상관 없을 때는 1을 반환
    if preference == '상관 없음':
        return 1
    
    #동갑 선호는 동갑인 사람의 조 내 비율을 계산
    elif preference == '동갑 선호':
        same_age_count = len(group[group['나이'] == row['나이']])
        return same_age_count / len(group)
    
    #연상 선호는 연상인 사람의 조 내 비율을 계산
    elif preference == '연상 선호':
        older_count = len(group[group['나이'] > row['나이']])
        return older_count / len(group)
    
    #연하 선호는 연하인 사람의 조 내 비율을 계산
    elif preference == '연하 선호':
        younger_count = len(group[group['나이'] < row['나이']])
        return younger_count / len(group)

def calculate_average_age_preference(test):
    #조 배치 총 내역 반환
    groups = test['조 배치'].unique()
    
    #조별 스코어를 계산할 list 
    group_preferences = []
    
    #각 조에 대해서 각 사람 별 나이대 배치 점수를 계산
    #조의 사람 별 나이대 배치 점수 평균을 계산
    for group in groups:
        group_df = test[test['조 배치'] == group]
        preferences = group_df.apply(calculate_age_preference, axis=1, group=group_df)
        group_preferences.append(preferences.mean())
        
    #전체 조에 대한 평균을 계산해 반환
    return np.mean(group_preferences)

'''선택 조건'''

#활동 선호
from collections import Counter

def calculate_activity_score(row, group):
    #활동 목록
    activities = ['맛집 탐방', '보드게임 카페', '방탈출', '노래방', 'PC방', '영화관', '한강 나들이', '카페/도서관 공부', '연극 / 뮤지컬 / 콘서트 등 공연 관람', '당구 / 볼링 등 스포츠']
    
    #조별 활동별 score를 계산 결과를 받을 dict 
    scores = {}
    
    #조원들의 활동별 선호를 계산. Counter를 이용해 True, False, None의 개수를 세기
    #None은 투표에 참여하지 않은 조원. 상관 없음으로 간주하고 전체 수에 계상
    #True와 False 중 Majority의 비율로 계산. 즉, True나 False 둘 중에 한 쪽으로 쏠릴 수록 점수가 높아짐.
    for activity in activities:
        preferences = group['희망 활동'].str.split(', ').apply(lambda x: activity in x if isinstance(x, list) else None)
        counter = Counter(preferences)
        if None in counter:
            none_count = counter.pop(None)
        else:
            none_count = 0
        if counter:
            majority_count = max(counter.values())
        else:
            majority_count = 0
        scores[activity] = (majority_count + none_count) / len(group)
        
    #반환되는 dict는 활동을 index로, 각 활동에 대한 majority 점수를 value로 가짐
    return scores


def calculate_average_activity_score(test):
    #조 배치 총 내역 반환
    groups = test['조 배치'].unique()
    
    #조별 스코어를 계산할 list 
    group_scores = []
    
    #각 조에 대해서 활동별 점수 dict를 반환
    #각 dict에 대해 활동별 점수 평균을 구해 group_scores에 쌓기
    for group in groups:
        group_df = test[test['조 배치'] == group]
        scores = group_df.apply(calculate_activity_score, axis=1, group=group_df)
        group_scores.append(np.mean([np.mean(list(score.values())) for score in scores]))
        
    #전체 조에 대한 평균을 계산해 반환
    return np.mean(group_scores)

def split_mbti(mbti):
    """
    주어진 MBTI 값을 각 성향에 따라 분할합니다.
    만약 MBTI 값이 없다면, 각 성향에는 NaN을 할당합니다.
    """
    if pd.isna(mbti):
        return pd.Series([np.nan] * 4)
    else:
        return pd.Series(list(mbti))
    
def calculate_mbti_preference(row, group):
    # E/I 희망
    preference_ei = row['E/I']

    # E_I가 빈 사람들의 수를 세기
    nan_ei_count = len(group[pd.isna(group['E_I'])])

    # 상관 없을 때는 1을 반환
    if preference_ei == '상관없음':
        score_ei = 1

    # 전자 선호는 E인 사람의 조 내 비율을 계산
    elif preference_ei == '전자선호':
        same_ei_count = len(group[group['E_I'] == 'E'])
        score_ei = (same_ei_count + nan_ei_count) / len(group)

    # 후자 선호는 I인 사람의 조 내 비율을 계산
    elif preference_ei == '후자선호':
        same_ei_count = len(group[group['E_I'] == 'I'])
        score_ei = (same_ei_count + nan_ei_count) / len(group)

    # N/S 희망
    preference_ns = row['N/S']

    # N_S가 빈 사람들의 수를 세기
    nan_ns_count = len(group[pd.isna(group['N_S'])])

    # 상관 없을 때는 1을 반환
    if preference_ns == '상관없음':
        score_ns = 1

    # 전자 선호는 N인 사람의 조 내 비율을 계산
    elif preference_ns == '전자선호':
        same_ns_count = len(group[group['N_S'] == 'N'])
        score_ns = (same_ns_count + nan_ns_count) / len(group)

    # 후자 선호는 S인 사람의 조 내 비율을 계산
    elif preference_ns == '후자선호':
        same_ns_count = len(group[group['N_S'] == 'S'])
        score_ns = (same_ns_count + nan_ns_count) / len(group)

    # F/T 희망
    preference_ft = row['F/T']

    # F_T가 빈 사람들의 수를 세기
    nan_ft_count = len(group[pd.isna(group['F_T'])])

    # 상관 없을 때는 1을 반환
    if preference_ft == '상관없음':
        score_ft = 1

    # 전자 선호는 F인 사람의 조 내 비율을 계산
    elif preference_ft == '전자선호':
        same_ft_count = len(group[group['F_T'] == 'F'])
        score_ft = (same_ft_count + nan_ft_count) / len(group)

    # 후자 선호는 T인 사람의 조 내 비율을 계산
    elif preference_ft == '후자선호':
        same_ft_count = len(group[group['F_T'] == 'T'])
        score_ft = (same_ft_count + nan_ft_count) / len(group)

    # J/P 희망
    preference_jp = row['J/P']

    # J_P가 빈 사람들의 수를 세기
    nan_jp_count = len(group[pd.isna(group['J_P'])])

    # 상관 없을 때는 1을 반환
    if preference_jp == '상관없음':
        score_jp = 1

    # 전자 선호는 J인 사람의 조 내 비율을 계산
    elif preference_jp == '전자선호':
        same_jp_count = len(group[group['J_P'] == 'J'])
        score_jp = (same_jp_count + nan_jp_count) / len(group)

    # 후자 선호는 P인 사람의 조 내 비율을 계산
    elif preference_jp == '후자선호':
        same_jp_count = len(group[group['J_P'] == 'P'])
        score_jp = (same_jp_count + nan_jp_count) / len(group)

    try:
        return (score_ei + score_ns + score_ft + score_jp) / 4
    except:
        return 1
    
def calculate_average_mbti_preference(test):
    test_temp = test.copy()
    
    # 'MBTI' 열을 분할하고 각 성향에 대한 새로운 열을 생성합니다.
    test_temp[['E_I', 'N_S', 'F_T', 'J_P']] = test_temp['MBTI'].apply(split_mbti)
    
    #조 배치 총 내역 반환
    groups = test_temp['조 배치'].unique()
    
    #조별 스코어를 계산할 list 
    group_preferences = []
    
    #각 조에 대해서 각 사람 별 MBTI 배치 점수를 계산
    #조의 사람 별 MBTI 배치 점수 평균을 계산
    for group in groups:
        group_df = test_temp[test_temp['조 배치'] == group]
        preferences = group_df.apply(calculate_mbti_preference, axis=1, group=group_df)
        group_preferences.append(preferences.mean())
        
    #전체 조에 대한 평균을 계산해 반환
    return np.mean(group_preferences)

In [8]:
import pandas as pd
import numpy as np
from random import shuffle, choice
from copy import deepcopy

# 초기 배정 함수
def initial_assignment(df):
    # 조 배정을 위한 초기 DataFrame 설정
    df['조 배치'] = np.nan

    # 신청자 수와 조 수 계산
    n = len(df)
    k = n // 6
    a = n % 6

    # 조 배치
    groups_6 = k - a
    groups_7 = a

    # 조 이름 생성
    group_names = [f'{i+1}' for i in range(groups_6 + groups_7)]

    # 6명 조와 7명 조로 나누기
    group_assign_6 = [6] * groups_6
    group_assign_7 = [7] * groups_7
    group_assign = group_assign_6 + group_assign_7

    # 조 배치 실행
    idx = 0
    for i, size in enumerate(group_assign):
        df.loc[idx:idx+size-1, '조 배치'] = group_names[i]
        idx += size

    return df


# 전역 변수로 원본 데이터 설정
raw_data_global = None

def set_global_raw_data(raw_data):
    global raw_data_global
    raw_data_global = raw_data
    
#평가 코드
def evaluate_assignment(test):
    global raw_data_global
    
    if check_missings(raw_data_global, test) and check_mapping(test) and check_group_num(test) and check_one_to_one(test):
       
        final_score = []
        
        leader_point = (check_leader_prefers(test) + check_possible_leaders(test)) / 2
        alcohol_point = check_alcohol_prefers(test)
        gender_ratio_point = calculate_gender_satisfying_ratio(raw, test)
        age_point = calculate_average_age_preference(test)
        
        final_score.append(leader_point)
        final_score.append(alcohol_point)
        final_score.append(gender_ratio_point)
        final_score.append(age_point)
        
        bonus_score = []
        
        activity_point = calculate_average_activity_score(test)
        mbti_point = calculate_average_mbti_preference(test)
        
        bonus_score.append(activity_point)
        bonus_score.append(mbti_point)
        
        final_point = np.mean(final_score) * 100
        bonus_point = np.mean(bonus_score) * 20

         # 최종 점수와 보너스 점수를 더해 반환
        return round(final_point + bonus_point, 2)
    else:
        return 0 # 기본 조건이 만족하지 않을 경우, 낮은 점수를 반환



# 시뮬레이티드 어닐링 알고리즘
def simulated_annealing(df, eval_func, T=1000, T_min=1, alpha=0.995):
    current_solution = df.copy()
    current_score = eval_func(current_solution)

    while T > T_min:
        # 무작위로 두 학생 선택
        student1, student2 = np.random.choice(df.index, 2, replace=False)

        # 두 학생의 조를 바꾸어 새로운 해를 생성
        new_solution = current_solution.copy()
        new_solution.loc[student1, '조 배치'], new_solution.loc[student2, '조 배치'] = \
        current_solution.loc[student2, '조 배치'], current_solution.loc[student1, '조 배치']

        # 새로운 해의 점수 계산
        new_score = eval_func(new_solution)

        # 점수가 높은 경우, 해를 업데이트
        if new_score > current_score:
            current_solution = new_solution
            current_score = new_score
        # 점수가 낮은 경우, 일정 확률로 해를 업데이트
        else:
            if np.random.rand() < np.exp((new_score - current_score) / T):
                current_solution = new_solution
                current_score = new_score

        # 온도 감소
        T *= alpha

    return current_solution


# 누락 학생 처리 함수
from random import choice

def handle_missing_students(df):
    missing_students = df[df['조 배치'].isna()]
    for idx, row in missing_students.iterrows():
        
        # 음주 호불호
        same_alcohol_group = df[df['음주 호불호'] == row['음주 호불호']]['조 배치'].dropna().unique()
        if len(same_alcohol_group) > 0:
            df.loc[idx, '조 배치'] = choice(same_alcohol_group)
            continue
        
        # 성비 고려 배치 (성비가 가장 균등한 조를 찾아 배치)
        gender_balance_group = df.groupby('조 배치')['성별'].value_counts().unstack().fillna(0)
        gender_balance_group['성비'] = abs(gender_balance_group['남'] - gender_balance_group['여'])
        best_group = gender_balance_group['성비'].idxmin()
        if pd.notna(best_group):
            df.loc[idx, '조 배치'] = best_group
            continue
        
        # 나이대 선호
        same_age_group = df[df['나이대 선호'] == row['나이대 선호']]['조 배치'].dropna().unique()
        if len(same_age_group) > 0:
            df.loc[idx, '조 배치'] = choice(same_age_group)
            continue
        
        # 활동 선호
        same_activity_group = df[df['활동 선호'] == row['활동 선호']]['조 배치'].dropna().unique()
        if len(same_activity_group) > 0:
            df.loc[idx, '조 배치'] = choice(same_activity_group)
            continue
        
        # MBTI
        same_mbti_group = df[df['MBTI'] == row['MBTI']]['조 배치'].dropna().unique()
        if len(same_mbti_group) > 0:
            df.loc[idx, '조 배치'] = choice(same_mbti_group)
            continue

        # 위의 모든 조건이 맞지 않는 경우, 랜덤 배치
        random_group = choice(df['조 배치'].dropna().unique())
        df.loc[idx, '조 배치'] = random_group

    return df




In [7]:
def main(file_name):
    # 원본 데이터 불러오기
    raw_data = pd.read_excel(file_name)
    
    # 초기 배정 수행
    initial_assignment_result = initial_assignment(raw_data)
    
    # 누락된 학생 처리
    handled_data = handle_missing_students(initial_assignment_result)
    
    # 원본 데이터를 전역 변수로 설정
    set_global_raw_data(raw_data)
    
    # 최적화 함수 호출
    final_assignment = simulated_annealing(handled_data, evaluate_assignment)
    
    # 결과 저장
    final_assignment.to_excel(f'JY_{file_name}', index=False)


In [5]:
main('Test_1.xlsx')

In [10]:
for i in range(1, 51):
    main(f'Test_{i}.xlsx')

In [28]:
if __name__ == "__main__":
    main()